In [1]:
1+1

2

In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
import os
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())

True


In [18]:
# 연결 상태 확인
print(client.is_ready())  # True가 출력되어야 함

# 메타데이터 조회
meta = client.get_meta()
print(meta)

True
{'grpcMaxMessageSize': 104858000, 'hostname': 'http://[::]:8080', 'modules': {'backup-gcs': {'bucketName': 'weaviate-wcs-prod-cust-asia-southeast1-workloads-backups', 'rootName': '0c042739-2a15-488c-8ba0-32a13000311f'}, 'generative-anthropic': {'documentationHref': 'https://docs.anthropic.com/en/api/getting-started', 'name': 'Generative Search - Anthropic'}, 'generative-anyscale': {'documentationHref': 'https://docs.anyscale.com/endpoints/overview', 'name': 'Generative Search - Anyscale'}, 'generative-aws': {'documentationHref': 'https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html', 'name': 'Generative Search - AWS'}, 'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/chat', 'name': 'Generative Search - Cohere'}, 'generative-databricks': {'documentationHref': 'https://docs.databricks.com/en/machine-learning/foundation-models/api-reference.html#completion-task', 'name': 'Generative Search - Databricks'}, 'generative-friendliai': {'documen

In [6]:
import weaviate.classes as wvc

# 간단한 컬렉션 생성
collection = client.collections.create(
    name="MyFirstCollection",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.config.Configure.Generative.openai(),
)

# 생성된 컬렉션 확인
print(client.collections.list_all())

c:\workdir\github-space-cap\GPT-study-blog\langchain_project\venv\Lib\site-packages\weaviate\warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(


{'MyFirstCollection': _CollectionConfigSimple(name='MyFirstCollection', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.OPENAI: 'generative-openai'>, model={}), properties=[], references=[], reranker_config=None, vectorizer_config=_VectorizerConfig(vectorizer=<Vectorizers.TEXT2VEC_OPENAI: 'text2vec-openai'>, model={'baseURL': 'https://api.openai.com', 'isAzure': False, 'model': 'text-embedding-3-small'}, vectorize_collection_name=True), vectorizer=<Vectorizers.TEXT2VEC_OPENAI: 'text2vec-openai'>, vector_config=None)}


In [9]:
import weaviate
import weaviate.classes.config as wc

# 컬렉션 생성 (올바른 방식)
collection = client.collections.create(
    name="Articles",
    vector_config=wc.Configure.Vectors.text2vec_openai(model="text-embedding-3-small"),
    generative_config=wc.Configure.Generative.openai(model="gpt-4o-mini"),
    properties=[
        wc.Property(name="title", data_type=wc.DataType.TEXT),
        wc.Property(name="content", data_type=wc.DataType.TEXT),
    ],
)

print(f"컬렉션 '{collection.name}' 생성 완료!")

컬렉션 'Articles' 생성 완료!


In [10]:
# 생성된 컬렉션 확인
print(client.collections.list_all())

{'Articles': _CollectionConfigSimple(name='Articles', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.OPENAI: 'generative-openai'>, model={'model': 'gpt-4o-mini'}), properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={'text2vec-openai': _PropertyVectorizerConfig(skip=False, vectorize_property_name=False)}), _Property(name='content', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={'text2vec-openai': _PropertyVectorizerConfig(skip=False, vectorize_property_name=False)})], references=[], reranker_config=None, vectorizer

In [ ]:
# 컬렉션 가져오기
my_collection = client.collections.get("MyFirstCollection")

# 단일 객체 추가
my_collection.data.insert(
    {
        "title": "첫 번째 문서",
        "content": "이것은 테스트 문서입니다.",
        "category": "example",
    }
)

# 여러 객체 일괄 추가
data_objects = [
    {"title": "문서 1", "content": "첫 번째 내용", "category": "tech"},
    {"title": "문서 2", "content": "두 번째 내용", "category": "science"},
    {"title": "문서 3", "content": "세 번째 내용", "category": "tech"},
]

my_collection.data.insert_many(data_objects)

UnexpectedStatusCodeError: Object was not added! Unexpected status code: 500, with response body: {'error': [{'message': 'vectorize target vector default: update vector: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY'}]}.

In [12]:
# 현재 컬렉션 목록 확인
print("삭제 전 컬렉션 목록:")
print(list(client.collections.list_all().keys()))

# 컬렉션 삭제
client.collections.delete("Articles")  # 이전에 생성한 "Articles" 컬렉션 삭제

print("삭제 후 컬렉션 목록:")
print(list(client.collections.list_all().keys()))

삭제 전 컬렉션 목록:
['Articles', 'MyFirstCollection']
삭제 후 컬렉션 목록:
['MyFirstCollection']


In [13]:
# 현재 컬렉션 목록 확인
print("삭제 전 컬렉션 목록:")
print(list(client.collections.list_all().keys()))

# 컬렉션 삭제
client.collections.delete("MyFirstCollection")  # 이전에 생성한 "Articles" 컬렉션 삭제

print("삭제 후 컬렉션 목록:")
print(list(client.collections.list_all().keys()))

삭제 전 컬렉션 목록:
['MyFirstCollection']
삭제 후 컬렉션 목록:
[]


In [14]:
# 컬렉션 생성 (최신 방식)
collection = client.collections.create(
    name="MyFirstCollection",
    vector_config=wc.Configure.Vectors.text2vec_openai(model="text-embedding-3-small"),
    generative_config=wc.Configure.Generative.openai(model="gpt-4o-mini"),
    properties=[
        wc.Property(name="title", data_type=wc.DataType.TEXT),
        wc.Property(name="content", data_type=wc.DataType.TEXT),
        wc.Property(name="category", data_type=wc.DataType.TEXT),
    ],
)

# 생성된 컬렉션 확인
print(f"컬렉션 '{collection.name}' 생성 완료!")
print("전체 컬렉션 목록:", list(client.collections.list_all().keys()))

컬렉션 'MyFirstCollection' 생성 완료!
전체 컬렉션 목록: ['MyFirstCollection']


In [19]:
# 컬렉션 가져오기
my_collection = client.collections.get("MyFirstCollection")

# 단일 객체 추가
my_collection.data.insert(
    {
        "title": "첫 번째 문서",
        "content": "이것은 테스트 문서입니다.",
        "category": "example",
    }
)

# 여러 객체 일괄 추가
data_objects = [
    {
        "title": "Python 기초",
        "content": "Python 프로그래밍 기초 내용",
        "category": "tech",
    },
    {
        "title": "AI와 머신러닝",
        "content": "인공지능과 기계학습 소개",
        "category": "science",
    },
    {
        "title": "데이터베이스 설계",
        "content": "효율적인 DB 설계 방법",
        "category": "tech",
    },
]

my_collection.data.insert_many(data_objects)
print(f"{len(data_objects)}개 데이터 추가 완료!")

UnexpectedStatusCodeError: Object was not added! Unexpected status code: 500, with response body: {'error': [{'message': 'vectorize target vector default: update vector: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY'}]}.